In [ ]:
# Audio Analysis
import os
import sys
print (os.getcwd())
sys.path.append('/home/Hanna.Drimalla/ml-autism/scripts')

In [ ]:
from __future__ import division

%matplotlib inline
import os
import subprocess
import numpy as np
import scipy
#import librosa
import matplotlib.pyplot as plt

import pandas as pd
import datetime as dt
import imageio

import sklearn
from sklearn.preprocessing import scale
import seaborn as sns
from scipy import stats

import mycharite
import mystats

In [ ]:
df, df_audio, action_r, action_c, gaze, audio =mycharite.load('charite')
df_audio['asc']=(df_audio.asd==11)*1

In [ ]:
df_audio=df_audio.rename(columns={'0' : 'spectrum_0', '1': 'spectrum_1', '2': 'spectrum_2', 
                      '3' : 'spectrum_3', '4': 'spectrum_4', '5': 'spectrum_5',
                      '6' : 'spectrum_6', '7': 'spectrum_7', '8': 'spectrum_8',
                      '9' : 'spectrum_9', '10': 'spectrum_10', '11': 'spectrum_11',
                      '12' : 'spectrum_12', '13': 'spectrum_13', '14': 'spectrum_14',
                      '15' : 'spectrum_15', '16': 'spectrum_16', '17': 'spectrum_17',
                      '18' : 'spectrum_18', '19': 'spectrum_19', '20': 'spectrum_20',
                      '21' : 'spectrum_21', '22': 'spectrum_22', '23': 'spectrum_23',
                      '24': 'spectrum_24', '25' : 'spectrum_25', '26': 'spectrum_26', 
                      '27': 'spectrum_27', '28' : 'spectrum_28', '29': 'spectrum_29',
                      '30': 'spectrum_30', '31' : 'spectrum_31', '32': 'spectrum_32', 
                      '33': 'spectrum_33',  '34' : 'spectrum_34', '35': 'spectrum_35',
                      '36': 'spectrum_36',  '37' : 'spectrum_37', '38': 'spectrum_38', 
                      '39': 'spectrum_39'                          
          })

In [ ]:
df_audio.columns
df_audio=df_audio.rename(columns={'disgust_proband': 'disgust_participant', 
                      'neutral_proband' : 'neutral_participant', 
                          'joy_proband': 'joy_participant'
                      })

df_audio=mycharite.adapt_times(df_audio)

In [ ]:
df_audio['participant_speaks']=(df_audio.conversation=='joy_participant') | (df_audio.conversation=='disgust_participant') | (df_audio.conversation=='neutral_proband')#if conversation==
df_audio=df_audio[(df_audio.participant_speaks==True)&(df_audio.conversation!='intro')].reset_index(drop=True)

In [ ]:
var=['pitch', 'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter',
   'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer',
   'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'JitterPCA', 'ShimmerPCA']


for i in var:
    print (i)
    print (df_audio.groupby('vpn').mean().groupby(['asc'])[i].describe())
    
    
var=['meanF0Hz', 'stdevF0Hz', 'HNR']


for i in var:
    print (i)
    print (df_audio.groupby('vpn').mean().groupby(['asc', 'sex_x'])[i].describe())

In [ ]:
def spectrogram_one(df_audio):
    data=np.array(df_audio.groupby('timestamp').mean()[audio].T)
    ax = sns.heatmap(data)
    plt.show()
    
def spectrogram(df_audio, name):
    plt.close()
    data=(df_audio.groupby('asc').mean()[audio[3:14]].T)
    #mfccs = sklearn.preprocessing.scale(data, axis=0)
    print (data)
    ax = sns.heatmap(np.array(data)) #, vmin=-2, vmax=2)
    plt.savefig('Heatmap' + name +'.png')
    plt.show()

In [ ]:
def group_comparison(df):
    var=['pitch','meanF0Hz', 'stdevF0Hz', 'HNR',
       'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter',
       'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer',
       'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'JitterPCA', 'ShimmerPCA']

    for i in var:
        print (i)
        df1=df[df.asc==0].groupby('vpn').mean()
        df2=df[df.asc==1].groupby('vpn').mean()
        mystats.two_ind_sample_tests(df1, df2, i)
        
group_comparison(df_audio[df_audio.conversation!='intro'])

In [ ]:
group_comparison(df_audio[df_audio.sex_x==1])

In [ ]:
group_comparison(df_audio[df_audio.sex_x==2])

In [ ]:
def group_comparison_median(df):
    var=['pitch','meanF0Hz', 'stdevF0Hz', 'HNR',
       'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter',
       'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer',
       'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'JitterPCA', 'ShimmerPCA']

    for i in var:
        print (i)
        df1=df[df.asc==0].groupby('vpn').median()
        df2=df[df.asc==1].groupby('vpn').median()
        mystats.two_ind_sample_tests(df1, df2, i)
        df1=df[df.asc==0].groupby('vpn').median()
        df2=df[df.asc==1].groupby('vpn').median()
        
group_comparison_median(df_audio[df_audio.conversation!='intro'])

In [ ]:
group_comparison_median(df_audio[df_audio.sex_x==1])

In [ ]:
group_comparison_median(df_audio[df_audio.sex_x==2])

In [ ]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
 
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov


In [ ]:
var=['pitch','meanF0Hz', 'stdevF0Hz', 'HNR',
       'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter',
       'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer',
       'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'JitterPCA', 'ShimmerPCA']

#PITCH IS ENERGY

formula = 'HNR ~ C(sex_x) + C(asc)'
model = ols(formula, df_audio.groupby('vpn').median()).fit()
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print(aov_table)


formula = 'meanF0Hz ~ C(sex_x) + C(asc)'
model = ols(formula, df_audio.groupby('vpn').mean()).fit()
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print(aov_table)


formula = 'stdevF0Hz ~ C(sex_x) + C(asc)'
model = ols(formula, df_audio.groupby('vpn').mean()).fit()
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print(aov_table)

formula = 'localabsoluteJitter ~ C(sex_x) + C(asc)'
model = ols(formula, df_audio.groupby('vpn').mean()).fit()
aov_table = anova_lm(model, typ=2)
eta_squared(aov_table)
omega_squared(aov_table)
print(aov_table)

In [ ]:
#df_audio['sex']=df_audio.sex_x

In [ ]:
a=0
for i in audio:
    print (i)
    plt.hist(((df_audio[df_audio.asc==0].groupby('vpn').mean()[i])))
    plt.show()


y, sr = librosa.load(audio_file, sr=None) // sr is none so librosa can sample the audion using the convenient sr
mfccs = librosa.feature.mfcc(y, sr, n_mfcc=13, hop_length=int(0.010sr), n_fft=int(0.025sr))
-To set the width of window to 25 ms you have tu multiply the sr and 0.025.The reason behind that is that sr is the number of samples in one second. so to get the number of samples in 25ms you have to multiply by 0.025.
-same thing for the stride, to get a stride of 10 ms, you need to multiply the sr and 0.010.